Source: https://platform.stratascratch.com/data-projects/data-transformation

In [2]:
import numpy as np
import pandas as pd

In [3]:
raw_json = pd.read_json('./datasets/case.json')

In [4]:
CurateOffer = raw_json.query("EventName =='CurateOffer_Result'")

In [5]:
fdsfd.test();lk
gfd





,EnqueuedTimeUtc,EventName,Payload
3,2021-08-25 05:02:55 UTC,CurateOffer_Result,"[{""curationProvider"":""ByPrice"",""offerId"":""149f..."
13,2021-08-25 09:03:29 UTC,CurateOffer_Result,"[{""curationProvider"":""ByPrice"",""offerId"":""c99a..."
15,2021-08-25 05:02:18 UTC,CurateOffer_Result,"[{""curationProvider"":""ByPrice"",""offerId"":""d668..."
17,2021-08-25 09:02:29 UTC,CurateOffer_Result,"[{""curationProvider"":""ByPrice"",""offerId"":""0a06..."
35,2021-08-25 09:03:14 UTC,CurateOffer_Result,"[{""curationProvider"":""ByPrice"",""offerId"":""135d..."


In [ ]:
fdsfd